In [1]:
from utils import *
from preprocessing import preprocess
from windowfy import windowfy
from featurizing import featurize
from training import train, do_ensemble, do_train
from eval_erisk import evaluate
from IPython.display import display, Markdown
from itertools import product
from numpy.random import seed
import tensorflow

/usr/local/lib/python3.6/site-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [2]:
seed(42)
tensorflow.random.set_seed(42) 
logger("Initialized numpy random and tensorflow random seed at 42")

Initialized numpy random and tensorflow random seed at 42


In [3]:
def do_experiment(params):
    logger("Starting experiment {}".format(params))
    windowfy(window_size=params["feat_window_size"], is_oversample=params["oversample"])
    featurize(calculate_feats=params["calculate_feats"], normalize=params["normalize"], discretize=params["discretize"],
                      include_feats=params["include_feats"])
    train(cnn_model=params["cnn_model"], maxlen=params["max_len"], batch_size=params["batch_size"])
    evaluate(params["eval_window_size"], params["feat_window_size"], params)
    logger("Finished experiment {}".format(params))

# obtain features and save them so we dont have to calculate them every single time
def do_experiment_first(params):
    windowfy(window_size=params["feat_window_size"], is_oversample=params["oversample"], 
             include_new_data=params["include_new_data"])
    featurize(calculate_feats=True, normalize=False, discretize=False, scale=False,
                      include_feats=params["include_feats"])

# obtain saved features, normalize or discretize them if necessary, and train and evaluate model
def do_experiment_second(params):
    featurize(calculate_feats=False, normalize=params["normalize"], discretize=params["discretize"],
                      scale=params["scale"], include_feats=params["include_feats"])
    do_train(model_name=params["model_name"], maxlen=params["max_len"], batch_size=params["batch_size"],
         shuffle=params["shuffle"])
    evaluate(params["eval_window_size"], params["feat_window_size"], params)
    
# obtain saved features, normalize or discretize them if necessary, and train and evaluate model
def do_experiment_second_old(params):
    featurize(calculate_feats=False, normalize=params["normalize"], discretize=params["discretize"],
                      scale=params["scale"], include_feats=params["include_feats"])
    train(cnn_model=True, maxlen=params["max_len"], batch_size=params["batch_size"],
         shuffle=params["shuffle"])
    evaluate(params["eval_window_size"], params["feat_window_size"], params)
    
    train(cnn_model=False, maxlen=params["max_len"], batch_size=params["batch_size"],
         shuffle=params["shuffle"])
    evaluate(params["eval_window_size"], params["feat_window_size"], params)
    
    do_ensemble(maxlen=params["max_len"], batch_size=params["batch_size"])
    evaluate(params["eval_window_size"], params["feat_window_size"], params)
    
    
def traverse(d):
    K,V = zip(*d.items())
    for v in product(*(v if isinstance(v,list) else traverse(v) for v in V)):
        yield dict(zip(K,v))
        
def plot_metrics(history):
    metrics = ['loss', 'prc', 'precision', 'recall']
    for n, metric in enumerate(metrics):
        name = metric.replace("_"," ").capitalize()
        plt.subplot(2,2,n+1)
        plt.plot(history.epoch, history.history[metric], color=colors[0], label='Train')
        plt.plot(history.epoch, history.history['val_'+metric],
                 color=colors[0], linestyle="--", label='Val')
        plt.xlabel('Epoch')
        plt.ylabel(name)
        if metric == 'loss':
          plt.ylim([0, plt.ylim()[1]])
        elif metric == 'auc':
          plt.ylim([0.8,1])
        else:
          plt.ylim([0,1])

        plt.legend()


## Experiments

In [4]:
first_part = {
    "include_feats": [["first_prons"],["first_prons","sentiment","nssi"]],
    "feat_window_size": [10000, 10], #10
    "oversample": [False, True],
    "include_new_data": [False, True]
}

second_part = {
    "scale": [False],
    "normalize": [False],
    "discretize": [False],
    "eval_window_size": [1],
    "max_len": [1000],
    "batch_size": [2], # 8, 2
    "shuffle": [True, False],
    "model_name": ["cnn_model", "lstm_model", "ensemble_model"]
}

In [5]:
firstpart_generator = traverse(first_part)

for i in firstpart_generator:
    try:
        logger("********** CALCULATING FEATURES FOR {} ***********".format(i))
        display(Markdown("#### Calculating features for {}".format(i)))
        do_experiment_first(i)

        secondpart_generator = traverse(second_part)

        for j in secondpart_generator:
            params = j.copy()
            params.update(i)
            logger("************ STARTING EXPERIMENT {} ***************".format(params))
            display(Markdown("#### Experiment {}".format(params)))
            try:
                do_experiment_second(params)
                logger("************ FINISHED EXPERIMENT {} ************* \n".format(params))
            except Exception as e:
                logger("*************************************")
                logger("Error during experiment {}: {}".format(params, e))
                logger("*************************************")
        del secondpart_generator
    except Exception as e:
        logger("*************************************")
        logger("General error during experiment {}: {}".format(i, e))
        logger("*************************************")

********** CALCULATING FEATURES FOR {'include_feats': ['first_prons'], 'feat_window_size': 10000, 'oversample': False, 'include_new_data': False} ***********


#### Calculating features for {'include_feats': ['first_prons'], 'feat_window_size': 10000, 'oversample': False, 'include_new_data': False}

Windowfying training users
[====================] 100%
Windowfying test users
[====================] 100%Data size: 340

Finished windowfying
Featurizing calculate_feats=True, normalize=False, discretize=False, discretize_size=10, include_feats=['first_prons']
Initialized numpy random and tensorflow random seed at 42
Data size: 340, 340
Data size: 423, 423
Calculating first prons
Calculating first prons
************ STARTING EXPERIMENT {'scale': False, 'normalize': False, 'discretize': False, 'eval_window_size': 1, 'max_len': 1000, 'batch_size': 2, 'shuffle': True, 'model_name': 'cnn_model', 'include_feats': ['first_prons'], 'feat_window_size': 10000, 'oversample': False, 'include_new_data': False} ***************


#### Experiment {'scale': False, 'normalize': False, 'discretize': False, 'eval_window_size': 1, 'max_len': 1000, 'batch_size': 2, 'shuffle': True, 'model_name': 'cnn_model', 'include_feats': ['first_prons'], 'feat_window_size': 10000, 'oversample': False, 'include_new_data': False}

Featurizing calculate_feats=False, normalize=False, discretize=False, discretize_size=10, include_feats=['first_prons']
Initialized numpy random and tensorflow random seed at 42
Starting training with cnn_model=cnn_model and maxlen=1000 and batch size=2
Generating embeddings
Data size: 340
Training
Epoch 1/100
20/20 [==============================] - 3s 62ms/step - loss: 28.2838 - tp: 1.0000 - fp: 4.0000 - tn: 32.0000 - fn: 3.0000 - accuracy: 0.8250 - precision: 0.2000 - recall: 0.2500 - auc: 0.5764 - prc: 0.1731 - val_loss: 104.2150 - val_tp: 0.0000e+00 - val_fp: 0.0000e+00 - val_tn: 55.0000 - val_fn: 13.0000 - val_accuracy: 0.8088 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_auc: 0.5000 - val_prc: 0.1912

Epoch 00001: val_prc improved from -inf to 0.19118, saving model to models/cnn_model10002True.hdf5
Epoch 2/100
20/20 [==============================] - 0s 11ms/step - loss: 36.9181 - tp: 0.0000e+00 - fp: 0.0000e+00 - tn: 35.0000 - fn: 5.0000 - accuracy: 0.8750 - precis

...

Featurizing calculate_feats=False, normalize=False, discretize=False, discretize_size=10, include_feats=['first_prons']
Initialized numpy random and tensorflow random seed at 42
Starting training with cnn_model=cnn_model and maxlen=1000 and batch size=2
Generating embeddings
Data size: 167714
Training
Epoch 1/100
20/20 [==============================] - 35s 2s/step - loss: 1.0291 - tp: 0.0000e+00 - fp: 0.0000e+00 - tn: 39.0000 - fn: 1.0000 - accuracy: 0.9750 - precision: 0.0000e+00 - recall: 0.0000e+00 - auc: 0.5000 - prc: 0.0250 - val_loss: 3.6941 - val_tp: 0.0000e+00 - val_fp: 0.0000e+00 - val_tn: 31436.0000 - val_fn: 2107.0000 - val_accuracy: 0.9372 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_auc: 0.4924 - val_prc: 0.0598

Epoch 00001: val_prc improved from -inf to 0.05981, saving model to models/cnn_model10002True.hdf5
Epoch 2/100
20/20 [==============================] - 31s 2s/step - loss: 1.2144 - tp: 0.0000e+00 - fp: 1.0000 - tn: 38.0000 - fn: 1.0000 - accuracy: 0


Epoch 00016: val_prc did not improve from 0.29414
Epoch 17/100
20/20 [==============================] - 31s 2s/step - loss: 0.0753 - tp: 0.0000e+00 - fp: 1.0000 - tn: 39.0000 - fn: 0.0000e+00 - accuracy: 0.9750 - precision: 0.0000e+00 - recall: 0.0000e+00 - auc: 0.0000e+00 - prc: 0.0000e+00 - val_loss: 3.7907 - val_tp: 0.0000e+00 - val_fp: 0.0000e+00 - val_tn: 31436.0000 - val_fn: 2107.0000 - val_accuracy: 0.9372 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_auc: 0.4962 - val_prc: 0.0610

Epoch 00017: val_prc did not improve from 0.29414
Epoch 18/100
20/20 [==============================] - 31s 2s/step - loss: 1.4419 - tp: 0.0000e+00 - fp: 0.0000e+00 - tn: 39.0000 - fn: 1.0000 - accuracy: 0.9750 - precision: 0.0000e+00 - recall: 0.0000e+00 - auc: 0.5000 - prc: 0.0250 - val_loss: 3.7106 - val_tp: 0.0000e+00 - val_fp: 0.0000e+00 - val_tn: 31436.0000 - val_fn: 2107.0000 - val_accuracy: 0.9372 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_auc: 0.4962 - val_prc: 0

#### Experiment {'scale': False, 'normalize': False, 'discretize': False, 'eval_window_size': 1, 'max_len': 1000, 'batch_size': 2, 'shuffle': True, 'model_name': 'lstm_model', 'include_feats': ['first_prons'], 'feat_window_size': 10, 'oversample': False, 'include_new_data': False}

Featurizing calculate_feats=False, normalize=False, discretize=False, discretize_size=10, include_feats=['first_prons']
Initialized numpy random and tensorflow random seed at 42
Starting training with cnn_model=lstm_model and maxlen=1000 and batch size=2
Generating embeddings
Data size: 167714
Training
Epoch 1/100
20/20 [==============================] - ETA: 0s - loss: 1.9977 - tp: 0.0000e+00 - fp: 7.0000 - tn: 32.0000 - fn: 1.0000 - accuracy: 0.8000 - precision: 0.0000e+00 - recall: 0.0000e+00 - auc: 0.3590 - prc: 0.0176

KeyboardInterrupt: 